# 🦌 Compounds 🧪

```python

```

In [ ]:
from collections import defaultdict
from dataclasses import dataclass, field
from inspect import isclass
from typing import ClassVar, Dict, List, Optional, Tuple, Union
from uuid import uuid4

import importnb

# import ipyelk.diagram.elk_export
import ipywidgets as W
import networkx as nx
from IPython.display import JSON, SVG, display

import ipyelk
import ipyelk.nx
import ipyelk.tools
import ipyelk.tools.tools

# from ipyelk.contrib.library import logic_gates as logic
from ipyelk import Elk
from ipyelk.contrib.elements import Compound, Edge, Label, Node, Port, Registry
from ipyelk.contrib.library import logic_gates
from ipyelk.contrib.shapes import connectors as conn
from ipyelk.contrib.shapes import shapes
from ipyelk.diagram import elk_export, elk_model
from ipyelk.diagram import layout_options as opt
from ipyelk.diagram import symbol
from ipyelk.diagram.defs import ConnectorDef, Def
from ipyelk.diagram.elk_model import strip_none
from ipyelk.diagram.layout_options.layout import ELKRectanglePacking
from ipyelk.diagram.symbol import Symbol
from ipyelk.transform import merge

### TODO

- how transformer deals with layoutOptions / properties / cssClasses
  - probably not great to have them in too many places...
  - "inherited" from higher nodes in the graph vs explicitly set in the node data

In [ ]:
n1 = Node(labels=[Label(text="hello")])

n1.add_child(Node())
n2 = Node(labels=[])
n2.a = Port()

n1.add_edge(n2.a, n1)

ilk = Compound()
ilk(n1)

In [ ]:
def nx_wrap(node: Node, context: Registry) -> Tuple[str, Node]:
    """Wrap the given node in another tuple so it can be used multiple times as
    a networkx node.

    :param node: Incomming Node Element to wrap
    :type node: Node
    :return: Tuple that describes this current node and context to be used in a
    networkx graph.
    :rtype: Tuple[str, Node]
    """
    return (context, node)


def get_children(node: Node):
    return getattr(node, "children", [])


@dataclass
class Compound:
    registry: Registry = field(default_factory=Registry)

    def _add(self, node, g, tree):
        context = self.registry
        with context:
            nx_node = nx_wrap(node, context)
            if nx_node not in g:
                g.add_node(nx_node, **node.to_json())

            for edge in getattr(node, "_edges", []):
                endpts = edge.points()
                nx_u, nx_v = map(lambda n: nx_wrap(n, context), endpts)
                for nx_pt, pt in zip([nx_u, nx_v], endpts):
                    if nx_pt not in g:
                        g.add_node(nx_pt, **pt.to_json())

                g.add_edge(nx_u, nx_v, **edge.to_json())

            for child in get_children(node):
                nx_child = self._add(child, g, tree)
                tree.add_edge(nx_node, nx_child)
            return nx_node

    def __call__(self, *nodes):
        g = nx.MultiDiGraph()
        tree = nx.DiGraph()
        for node in nodes:
            self._add(node, g, tree)
        return (g, tree)

In [ ]:
# configure app
app = Elk(
    transformer=ipyelk.nx.XELK(
        source=ilk(n1),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

### mutating the compound and updating the transformer

In [ ]:
ng = Node(shape=logic_gates.Nand_Gate())
nor = Node(shape=logic_gates.Nor_Gate())
n1.add_child(ng)

n1.add_child(nor)

app.diagram.defs = logic_gates.Gate.make_defs()
app.transformer.source = ilk(n1)

### Adding edge

In [ ]:
c2 = n1.add_edge(ng.b, ng.out)
app.transformer.source = ilk(n1)

In [ ]:
c2 = ng.add_edge(nor.b, ng.a)
app.transformer.source = ilk(n1)

In [ ]:
JSON(
    [
        {"u": str(u), "v": str(v), **d}
        for u, v, d in app.transformer.source[0].edges(data=True)
    ]
)

In [ ]:
# ng.shape = logic_gates.Nand_Gate()
ng.shape = logic_gates.Nor_Gate()
app.transformer.source = ilk(n1)

In [ ]:
from IPython.display import JSON

JSON([d for *_, d in app.transformer.source[0].edges(data=True)])

# Experiments for Record Nodes

In [ ]:
app = Elk(
    transformer=ipyelk.nx.XELK(
        #                 layouts={
        # #                     elk_model.ElkRoot: {
        # #                         "parents": opt.OptionsWidget(options=[opt.HierarchyHandling()]).value,
        # #                     },
        #                 },
        #         source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app

In [ ]:
from dataclasses import dataclass, field
from typing import Dict

record_opts = opt.OptionsWidget(
    options=[
        #         opt.LayoutAlgorithm(value=ELKRectanglePacking.identifier),
        opt.HierarchyHandling(),
        opt.Padding(left=0, right=0, bottom=0, top=0),
        opt.NodeSpacing(spacing=0),
        opt.EdgeNodeSpacing(spacing=0),
        opt.AspectRatio(ratio=100),
        opt.ExpandNodes(activate=True),
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        opt.NodeSizeConstraints(),
        opt.ComponentsSpacing(spacing=0),
        opt.NodeSpacing(spacing=0),
    ]
).value

label_opts = opt.OptionsWidget(
    options=[
        opt.NodeLabelPlacement(horizontal="center", vertical="center"),
        opt.LabelSpacing(spacing=10),
    ]
).value


@dataclass
class Record(Node):
    layoutOptions: Dict = field(default_factory=lambda: {**record_opts})
    width: float = field(default=80)

    def __hash__(self):
        return hash(id(self))

    def to_json(self):
        for child in self.children:
            child.layoutOptions = merge(
                opt.OptionsWidget(
                    options=[
                        opt.NodeSizeConstraints(),
                        opt.NodeSizeMinimum(width=self.width, height=20),
                    ]
                ).value,
                child.layoutOptions,
            )
        return super().to_json()

In [ ]:
r = Record(
    #     labels=[Label(text="Class")]
    #     shape=symbol.Rect(width=150),
    width=180
)


def make_labels(texts):
    return [
        Label(
            text=t,
            layoutOptions=label_opts,
        )
        for t in texts
    ]


r.add_child(
    Node(
        labels=make_labels(["methods", "subtitle"]),
        #     layoutOptions=compartment_opts,
        #     shape=symbol.Rect(width=60, height=20)
    )
)

r.add_child(n1)

cp = Compound()
# cp.add_node(r)
# cp.add_node(n1)

r.width = 300

app.diagram.defs = logic_gates.Gate.make_defs()
app.transformer.source = cp(r)

In [ ]:
display(JSON(app.diagram.mark_layout))

# Label Icons

In [ ]:
conversation_icon = Def(
    children=[
        symbol.SVG(
            value="""<path d="M24,17H8a1,1,0,0,0,0,2H24a1,1,0,0,0,0-2Z"/><path d="M24,7H8A1,1,0,0,0,8,9H24a1,1,0,0,0,0-2Z"/><path d="M24,12H8a1,1,0,0,0,0,2H24a1,1,0,0,0,0-2Z"/><path d="M25,2H7A5,5,0,0,0,2,7V27.11a3,3,0,0,0,3,3,3,3,0,0,0,1.75-.56l6.81-4.87A3,3,0,0,1,15.45,24H25a5,5,0,0,0,5-5V7A5,5,0,0,0,25,2Zm3,17a3,3,0,0,1-3,3H15.45a4.94,4.94,0,0,0-3.11,1.09L5.58,27.92a1,1,0,0,1-1,.08A1,1,0,0,1,4,27.11V7A3,3,0,0,1,7,4H25a3,3,0,0,1,3,3Z"/>"""
        )
    ],
    width=32,
    height=32,
)
icons = {"conversation": conversation_icon}

In [ ]:
app = Elk(
    transformer=ipyelk.nx.XELK(
        #                 layouts={
        # #                     elk_model.ElkRoot: {
        # #                         "parents": opt.OptionsWidget(options=[opt.HierarchyHandling()]).value,
        # #                     },
        #                 },
        #         source=(g, tree),
    ),
    layout={"height": "100%"},
    style={
        " .hidden": {
            #             "display": "none",
        }
    },
)
app.diagram.defs = icons
app

In [ ]:
await app.transformer._refresh()

In [ ]:
%debug

In [ ]:
@dataclass
class Icon(Symbol):
    type = "label:icon"
    value: str = " "

    def get_shape_props(self):
        props = super().get_shape_props()
        if self.value:
            props.update({"use": str(self.value)})
        return props


icon = Icon(value="conversation", width=20, height=50)
l1 = Label(shape=icon, layoutOptions=label_opts)
l2 = Label(text="something to talk about", layoutOptions=label_opts)
cn = Node(
    labels=[
        l1,
        l2,
    ]
)
cmp = Compound()
app.transformer.source = cmp(cn)

In [ ]:
icon.height = 40
cn.labels = [l2]
l2.labels = [l1]
app.transformer.source = cmp(cn)

In [ ]:
cn = Node(
    labels=[
        Label(
            text="something to talk about",
            layoutOptions=label_opts,
            labels=[
                Label(shape=icon, layoutOptions=label_opts),
            ],
        ),
    ]
)
# cmp = Compound()
app.transformer.source = cmp(cn)

In [ ]:
icon.height = 40
cn.labels = [l1, l2]
l2.labels = []
app.transformer.source = cmp(cn)

In [ ]:
app.transformer.value